In [0]:
!pip install psycopg2-binary

In [0]:
import psycopg2

In [0]:
dir(psycopg2)

In [0]:
'''psycopg2.connect looks like it may be interesting! 
  (Similar to how sqlite3 module worked)'''
help(psycopg2.connect)

In [0]:
dbname = 'gqblsofi'
user = 'gqblsofi'
password = '1234'
host = 'rajje.db.elephantsql.com'

In [0]:
pg_conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host)

In [0]:
pg_conn

<connection object at 0x7f0f1d3a2c78; dsn: 'user=gqblsofi password=xxx dbname=gqblsofi host=rajje.db.elephantsql.com', closed: 0>

In [0]:
dir(pg_conn)

In [0]:
pg_curs = pg_conn.cursor()

In [0]:
help(pg_curs.execute)

Help on built-in function execute:

execute(...) method of psycopg2.extensions.cursor instance
    execute(query, vars=None) -- Execute query with bound vars.



In [0]:
create_table_statement = """
CREATE TABLE test_table (
  id        SERIAL PRIMARY KEY,
  name  varchar(40) NOT NULL,
  data    JSONB
);
"""

pg_curs.execute(create_table_statement)
pg_conn.commit()

In [0]:
insert_statement = """
INSERT INTO test_table (name, data) VALUES
(
  'A row name',
  null
),
(
  'Another row, with JSON',
  '{ "a": 1, "b": ["dog", "cat", 42], "c": true }'::JSONB
);
"""

pg_curs.execute(insert_statement)
pg_conn.commit()

In [0]:
query = "SELECT * FROM test_table;"
pg_curs.execute(query)
pg_curs.fetchall()

[(1, 'A row name', None),
 (2, 'Another row, with JSON', {'a': 1, 'b': ['dog', 'cat', 42], 'c': True})]

# ETL - RPG data from SQLite to PostgreSQL

We'd like to get the RPG data out of SQLite and insert it into PostgreSQL.

Aka - we're making a data pipeline! Aka - an ETL (Extract Transform Load). Our first "cloud" ETL!

In [0]:
!wget https://github.com/KryssyCo/DS-Unit-3-Sprint-2-SQL-and-Databases/blob/master/module1-introduction-to-sql/rpg_db.sqlite3?raw=true

--2020-02-04 00:46:52--  https://github.com/KryssyCo/DS-Unit-3-Sprint-2-SQL-and-Databases/blob/master/module1-introduction-to-sql/rpg_db.sqlite3?raw=true
Resolving github.com (github.com)... 192.30.253.112
Connecting to github.com (github.com)|192.30.253.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/KryssyCo/DS-Unit-3-Sprint-2-SQL-and-Databases/raw/master/module1-introduction-to-sql/rpg_db.sqlite3 [following]
--2020-02-04 00:46:52--  https://github.com/KryssyCo/DS-Unit-3-Sprint-2-SQL-and-Databases/raw/master/module1-introduction-to-sql/rpg_db.sqlite3
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/KryssyCo/DS-Unit-3-Sprint-2-SQL-and-Databases/master/module1-introduction-to-sql/rpg_db.sqlite3 [following]
--2020-02-04 00:46:52--  https://raw.githubusercontent.com/KryssyCo/DS-Unit-3-Sprint-2-SQL-and-Databases/master/module1-introduction-to-sq

In [0]:
!mv 'rpg_db.sqlite3?raw=true' rpg_db.sqlite3

In [0]:
!ls

rpg_db.sqlite3	sample_data


In [0]:
import sqlite3
sl_conn = sqlite3.connect('rpg_db.sqlite3')
sl_curs =sl_conn.cursor()

In [0]:
# We care about the charactercreator_character table
row_count = 'SELECT COUNT(*) FROM charactercreator_character'
sl_curs.execute(row_count).fetchall()

[(302,)]

In [0]:
# Our goal - copy the characters table from SQLite to PostgreSQL using Python
# Step 1 - E = Extract: Get the characters from the table
get_characters = 'SELECT * FROM charactercreator_character'
characters = sl_curs.execute(get_characters).fetchall()

In [0]:
characters[:5]

[(1, 'Aliquid iste optio reiciendi', 0, 0, 10, 1, 1, 1, 1),
 (2, 'Optio dolorem ex a', 0, 0, 10, 1, 1, 1, 1),
 (3, 'Minus c', 0, 0, 10, 1, 1, 1, 1),
 (4, 'Sit ut repr', 0, 0, 10, 1, 1, 1, 1),
 (5, 'At id recusandae expl', 0, 0, 10, 1, 1, 1, 1)]

In [0]:
len(characters)

302

In [0]:
# Step 2 - Transform
# In this case, we don't actually want/need to change much
# Because we want to keep all the data
# And we're going from SQL to SQL

# But what do we need to be able to load into PostgreSQL
# We need to make a new table with the apprropriate schema

# What was the old schema? We can get at this with SQLitte internals
sl_curs.execute('PRAGMA table_info(charactercreator_character);').fetchall()

[(0, 'character_id', 'integer', 1, None, 1),
 (1, 'name', 'varchar(30)', 1, None, 0),
 (2, 'level', 'integer', 1, None, 0),
 (3, 'exp', 'integer', 1, None, 0),
 (4, 'hp', 'integer', 1, None, 0),
 (5, 'strength', 'integer', 1, None, 0),
 (6, 'intelligence', 'integer', 1, None, 0),
 (7, 'dexterity', 'integer', 1, None, 0),
 (8, 'wisdom', 'integer', 1, None, 0)]

In [0]:
create_character_table = """
CREATE TABLE charactercreator_character (
  character_id SERIAL PRIMARY KEY,
  name VARCHAR(30),
  level INT,
  exp INT,
  hp INT,
  strength INT,
  intelligence INT,
  dexterity INT,
  wisdom INT
);
"""

In [0]:
pg_curs.execute(create_character_table)
pg_conn.commit()

In [0]:
# We can query tables if we want to check
# This is a clever optional thing, showing postgresql internals
show_tables = """
SELECT
   *
FROM
   pg_catalog.pg_tables
WHERE
   schemaname != 'pg_catalog'
AND schemaname != 'information_schema';
"""
pg_curs.execute(show_tables)
pg_curs.fetchall()

[('public', 'test_table', 'gqblsofi', None, True, False, False, False),
 ('public',
  'charactercreator_character',
  'gqblsofi',
  None,
  True,
  False,
  False,
  False)]

In [0]:
characters[0]

(1, 'Aliquid iste optio reiciendi', 0, 0, 10, 1, 1, 1, 1)

In [0]:
example_insert = """
INSERT INTO charactercreator_character
(name, level, exp, hp, strength, intelligence, dexterity, wisdom)
VALUES """ +str(characters[0][1:]) + ";"

print(example_insert)


INSERT INTO charactercreator_character
(name, level, exp, hp, strength, intelligence, dexterity, wisdom)
VALUES ('Aliquid iste optio reiciendi', 0, 0, 10, 1, 1, 1, 1);


In [0]:
# How do we do this for all characters? Loops!
for character in characters:
  insert_character = """
    INSERT INTO charactercreator_character
    (name, level, exp, hp, strength, intelligence, dexterity, wisdom)
    VALUES """ + str(character[1:]) + ";"
  pg_curs.execute(insert_character)
# pg_conn.commit()


In [0]:
pg_curs.execute('SELECT * FROM charactercreator_character')
pg_curs.fetchall()

[(1, 'Aliquid iste optio reiciendi', 0, 0, 10, 1, 1, 1, 1),
 (2, 'Optio dolorem ex a', 0, 0, 10, 1, 1, 1, 1),
 (3, 'Minus c', 0, 0, 10, 1, 1, 1, 1),
 (4, 'Sit ut repr', 0, 0, 10, 1, 1, 1, 1),
 (5, 'At id recusandae expl', 0, 0, 10, 1, 1, 1, 1),
 (6, 'Non nobis et of', 0, 0, 10, 1, 1, 1, 1),
 (7, 'Perferendis', 0, 0, 10, 1, 1, 1, 1),
 (8, 'Accusantium amet quidem eve', 0, 0, 10, 1, 1, 1, 1),
 (9, 'Sed nostrum inventore error m', 0, 0, 10, 1, 1, 1, 1),
 (10, 'Harum repellendus omnis od', 0, 0, 10, 1, 1, 1, 1),
 (11, 'Itaque ut commodi,', 0, 0, 10, 1, 1, 1, 1),
 (12, 'Molestiae quis', 0, 0, 10, 1, 1, 1, 1),
 (13, 'Ali', 0, 0, 10, 1, 1, 1, 1),
 (14, 'Tempora quod optio possimus il', 0, 0, 10, 1, 1, 1, 1),
 (15, 'Sed itaque beatae pari', 0, 0, 10, 1, 1, 1, 1),
 (16, 'Quam dolor', 0, 0, 10, 1, 1, 1, 1),
 (17, 'Molestias expedita', 0, 0, 10, 1, 1, 1, 1),
 (18, 'Lauda', 0, 0, 10, 1, 1, 1, 1),
 (19, 'Incidunt sint perferen', 0, 0, 10, 1, 1, 1, 1),
 (20, 'Laboriosa', 0, 0, 10, 1, 1, 1, 1),
 (21,

In [0]:
pg_conn.commit()

In [0]:
# A quick test that we did this correctly
pg_curs.execute('SELECT * FROM charactercreator_character')
pg_characters = pg_curs.fetchall()

In [0]:
characters[0]

(1, 'Aliquid iste optio reiciendi', 0, 0, 10, 1, 1, 1, 1)

In [0]:
pg_characters[0]

(1, 'Aliquid iste optio reiciendi', 0, 0, 10, 1, 1, 1, 1)

In [0]:
for character, pg_character in zip(characters, pg_characters):
  assert character == pg_character